#Machine Translation with Trax

Note by Denis Rothman: The original notebook was split into cells.

[Reference Code](https://colab.research.google.com/github/google/trax/blob/master/trax/intro.ipynb)


#Installing Trax

In [ ]:
#@title Installing Trax
import os
import numpy as np

!pip install -q -U trax
import trax

     |████████████████████████████████| 637 kB 13.3 MB/s 
     |████████████████████████████████| 4.9 MB 45.1 MB/s 
     |████████████████████████████████| 462 kB 49.8 MB/s 


#Creating a tranformer model

In [ ]:
#@title Creating a tranformer model
# Pre-trained model config in gs://trax-ml/models/translation/ende_wmt32k.gin
model = trax.models.Transformer(
    input_vocab_size=33300,
    d_model=512, d_ff=2048,
    n_heads=8, n_encoder_layers=6, n_decoder_layers=6,
    max_len=2048, mode='predict')


#Initializing the model using pre-trained weights

In [ ]:
#@title Initializing the model using pre-trained weights
model.init_from_file('gs://trax-ml/models/translation/ende_wmt32k.pkl.gz',
                     weights_only=True)

(((),
  ((), ((), ((), (), ()))),
  ((array([[ 0.05705298,  0.0897216 ,  0.02124788, ..., -0.07733061,
            -0.00756849,  0.0863874 ],
           [-0.6957528 ,  0.52017885,  0.01700701, ..., -0.12895925,
             0.35437337, -0.9630883 ],
           [ 0.05140841, -0.02195717, -0.09755829, ...,  0.08997008,
             0.13417095, -0.26320305],
           ...,
           [ 0.10300644, -0.06085985, -0.27162302, ...,  0.05554791,
            -0.11951675,  0.17515816],
           [-0.03211264,  0.01250579, -0.7359762 , ...,  0.39658797,
             0.6097316 ,  0.38914362],
           [-0.1319148 ,  0.13255426, -0.2606699 , ...,  0.02071916,
             0.1670416 ,  0.03873056]], dtype=float32),
    (),
    array([[[ 1.143651  ,  1.0403643 ,  0.1692191 , ...,  0.4217065 ,
             -0.3508536 ,  0.24478772],
            [ 0.8617227 , -0.77985394,  1.7588066 , ...,  0.11070859,
              0.2525967 ,  0.11612047],
            [-0.14974353, -0.88516223,  0.0342847 , ..., 

#Tokenizing a sentence

In [ ]:
#@title Tokenizing a sentence
sentence = 'I am only a machine but I have machine intelligence.'

tokenized = list(trax.data.tokenize(iter([sentence]),  # Operates on streams.
                                    vocab_dir='gs://trax-ml/vocabs/',
                                    vocab_file='ende_32k.subword'))[0]


#Decoding from the Transformer

In [ ]:
#@title Decoding from the Transformer
tokenized = tokenized[None, :]  # Add batch dimension.
tokenized_translation = trax.supervised.decoding.autoregressive_sample(
    model, tokenized, temperature=0.0)  # Higher temperature: more diverse results.


#De-tokenizing and Displaying the Translation

In [ ]:
#@title De-tokenizing and Displaying the Translation
tokenized_translation = tokenized_translation[0][:-1]  # Remove batch and EOS.
translation = trax.data.detokenize(tokenized_translation,
                                   vocab_dir='gs://trax-ml/vocabs/',
                                   vocab_file='ende_32k.subword')
print("The sentence:",sentence)
print("The translation:",translation)

The sentence: I am only a machine but I have machine intelligence.
The translation: Ich bin nur eine Maschine, aber ich habe Maschinenübersicht.
